In [1]:
import collections
import copy

class M:
    idgen=0
    
    #get op and modes of following 3 addresses
    def ix(s,x):
        assert(x>=0)
        x="%05d"%(x)
        op=int(x[3:])
        m3,m2,m1=[int(c) for c in x[:3]]
        return op,m1,m2,m3


    def gv(s,m,ix):
        assert(ix in [1,2,3])
        if m==0:#reg
            return s.p[s.p[s.ip+ix]]
        elif m==1:#imm
            return s.p[s.ip+ix] 
        elif m==2:#reg+rb
            return s.p[s.p[s.ip+ix]+s.rb]
        assert(0)
        
    def __init__(s,p):
        #load to mem
        s.p=collections.defaultdict(int)
        for i,v in enumerate(p):
            s.p[i]=v
            
        
        #input queue
        s.i=[]
        
        #id
        s.id=M.idgen
        M.idgen+=1
        
        s.ip=0
        
        #print("M",s.id,"created")
        
        #relative base
        s.rb=0
        
        #count all vm ticks
        s.ticks=0
        
        s.halt=False
        
        s.st=0
        s.tgt=0
        
        s.idle=0
    
    
    def sendLine(s, l):
        for c in l:
            s.i.append(ord(c))
        s.i.append(ord('\n'))
        
    def sendInput(s, i):
        s.i.append(i)
        
     
    def tick(s):
        r=None
        
        if s.halt:
            print("halted")
            return r
        op,m1,m2,m3=s.ix(s.p[s.ip])
        if op==1:#add
            assert(m3 in [0,2])
            tix=s.p[s.ip+3]
            if m3==2:
                tix+=s.rb
            s.p[tix]=s.gv(m1,1)+s.gv(m2,2)
            s.ip+=4
        elif op==2:#mul
            assert(m3 in [0,2])
            tix=s.p[s.ip+3]
            if m3==2:
                tix+=s.rb
            s.p[tix]=s.gv(m1,1)*s.gv(m2,2)
            s.ip+=4
        elif op==3:#in
            assert(m1 in [0,2])
            iv=None
            if len(s.i)>0:
                iv=s.i[0]
                s.i=s.i[1:]
            else:
                iv=-1
                s.idle+=1
            tix=s.p[s.ip+1]
            if m1==2:
                tix+=s.rb
            s.p[tix]=iv
            s.ip+=2
        elif op==4:#out
            r=s.gv(m1,1)
            #print(s.id,"spits out",r)
            s.ip+=2
            s.idle=0
            #print("--id:",s.id,"onwards")
        elif op==5:#jump-if-true
            if s.gv(m1,1):
                s.ip=s.gv(m2,2)
            else:
                s.ip+=3
        elif op==6:#jump-if-false
            if not s.gv(m1,1):
                s.ip=s.gv(m2,2)
            else:
                s.ip+=3
        elif op==7:#less-than
            assert(m3 in [0,2])
            tix=s.p[s.ip+3]
            if m3==2:
                tix+=s.rb
            s.p[tix]=s.gv(m1,1)<s.gv(m2,2)
            s.ip+=4
        elif op==8:#equals
            assert(m3 in [0,2])
            tix=s.p[s.ip+3]
            if m3==2:
                tix+=s.rb
            s.p[tix]=s.gv(m1,1)==s.gv(m2,2)
            s.ip+=4
        elif op==9:#adjust relative base
            s.rb+=s.gv(m1,1)
            s.ip+=2
        elif op==99:#end
            s.halt = True
            #print("\nhalted", s.id)
            r=None
        else:
            #pass
            print("id", s.id, "op",op)
            assert(0)
        #print(op)
        s.ticks+=1
        return r
    
    def process(s,i=None):
        
        if not i is None:
            s.sendLine(i)
            
        o=""
        ticks=0
        while not 'Command?\n' in o and not s.halt:
            r = s.tick()
            if not r is None:
                o+=chr(r)
            
            if ticks >100e3:
                assert(0)
            ticks += 1
        return o
        
def ld():
    return [int(x) for x in open("25.txt").read().split(',')]
      
    


In [2]:
#interactive
"""
m=M(ld())

path=[]
while 1:
    z=m.process()
    print(path)
    print(z)
    iii=input()
    m.sendLine(iii)
    path.append(iii)
"""  
    
#monolith
#fuel cell
#astrolobe
#klein bottle
#dark matter
#tambourine




'\nm=M(ld())\n\npath=[]\nwhile 1:\n    z=m.process()\n    print(path)\n    print(z)\n    iii=input()\n    m.sendLine(iii)\n    path.append(iii)\n'

In [3]:
#ised interactive mode above to explore
pl="west,take klein bottle,"
pl+="south,east,take monolith,"
pl+="south,take fuel cell,west,"
pl+="west,take astrolabe,east,"
pl+="east,north,west,north,"
pl+="west,north,take tambourine,"
pl+="south,west,take dark matter,west,"
pl+="inv"

pl=pl.split(',')

m=M(ld())
z=m.process()#print(z)
for s in pl:
    z=m.process(s)
    #print(z)

    
def opt(z):
    r=[]
    for l in z.split('\n'):
        if l[:2]=="- ":
            r.append(l[2:])
    return r
            

items=opt(m.process("inv"))
#print(items)

for i in range(pow(2,len(items))):
    tst=[]
    for j in range(len(items)):
        if 1<<j & i:
            tst.append(items[j])
    #print(tst)
    mm=copy.deepcopy(m)
    for j in tst:
        z=mm.process("drop %s"%(j))
    z=mm.process("north")
    if not "back to the checkpoint" in z:
        for w in z.split(' '):
            if w.isnumeric():
                print("part1:",w,67635328)
            
        


part1: 67635328 67635328
